# 株分析のプログラム集

## 1.株価を取得する、

In [2]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import datetime
import os
import time

#チャートグラフ関係
from math import pi
from bokeh.plotting import figure  ,output_notebook , show, output_file
from bokeh.layouts import column
from bokeh.models import LinearAxis, Range1d

#銘柄コードのリストを取得する
df_list = pd.read_excel('input/codelist.xlsx' ,index_col=None)
print(df_list.dtypes)
#銘柄コードが数値なら文字列に変更
if df_list['code'].dtype=='int64':
    df_list['code']=df_list['code'].astype('str')
lists=df_list['code']
for securitycode in lists:
    #securitycode:銘柄コード
    print(securitycode)
    #開始日
    startdate='1/1/2010'#mm/dd/YYYY
    #終了日(今日)
    enddate=datetime.date.today().strftime('%m/%d/%Y')
    #株価データcsvファイルが存在するか確認する。
    #ファイルパス
    FilePath_CSV='env/株価_'+securitycode+'.csv'
    #最新株価が取得できているか判定する。
    if os.path.isfile(FilePath_CSV):
        df=pd.read_csv(FilePath_CSV)
        date_stock=datetime.date.today()
        #今日が土日の場合は直近の木曜日にする。平日の場合は前日にする。
        if date_stock.weekday()>=5:
            date_stock=datetime.date.today()-datetime.timedelta(days=date_stock.weekday()-3)
        else:
            date_stock=datetime.date.today()-datetime.timedelta(days=1)
        df['Date'] = pd.to_datetime(df['Date'])
        #当日もしくは直近のデータを抜き取る。
        df2=df[df["Date"] >= datetime.datetime(date_stock.year,date_stock.month,date_stock.day)]
        #株価データが存在している場合
        if len(df2)>=1:
            print('yes')
            df_stock = df.set_index('Date')
        else:#株価データがない場合
            #株価を取得する（日足）
            df_stock =pdr.stooq.StooqDailyReader(symbols=securitycode+'.jp'
                                            , start=startdate, end=enddate).read().sort_values(by='Date',ascending=True)
            #株価を保存する。
            df_stock.to_csv(FilePath_CSV)
    else:#株価データがない場合
        #株価を取得する（日足）
        df_stock =pdr.stooq.StooqDailyReader(symbols=securitycode+'.jp'
                                        , start=startdate, end=enddate).read().sort_values(by='Date',ascending=True)
        #株価を保存する。
        df_stock.to_csv(FilePath_CSV)

    #print(df_stock)
    #移動平均を計算する
    #50移動平均,100移動平均,200移動平均
    MA_list=[50,100,200]
    for MA in MA_list:
        df_stock['MA'+str(MA)+'d']=df_stock['Close'].rolling(MA).mean()
    #移動平均乖離率
    df_stock['KairiMA'+str(MA_list[0])+'d'] = (df_stock['Close'] - df_stock['MA'+str(MA_list[0])+'d'])/df_stock['MA'+str(MA_list[0])+'d']*100

    #ボリンジャーバンド2シグマを計算する。
    BBspan=20
    #ボリンジャーバンドを計算
    df_stock['BBpluse']=df_stock['Close'].rolling(BBspan).mean()+df_stock['Close'].rolling(BBspan).std()*2
    df_stock['BBminus']=df_stock['Close'].rolling(BBspan).mean()-df_stock['Close'].rolling(BBspan).std()*2

    #RSIを計算する
    rsi_kikan=14
    df_stock['rsi_up']=df_stock['Close'].diff()
    df_stock['rsi_down']=-1*df_stock['rsi_up']
    df_stock.rsi_up[df_stock.rsi_up < 0]=0
    df_stock.rsi_down[df_stock.rsi_down < 0]=0
    df_stock['rsi']=df_stock.rsi_up.rolling(rsi_kikan).mean()/(df_stock.rsi_up.rolling(rsi_kikan).mean()
                                                  +df_stock.rsi_down.rolling(rsi_kikan).mean())*100
    #MACDを計算する。
    EMA_list=[8,18]
    Signal=6
    for EMA in EMA_list:
        df_stock['EMA'+str(EMA)]=df_stock['Close'].ewm(span=EMA).mean()
    df_stock['MACD'] = df_stock['EMA'+str(EMA_list[0])] - df_stock['EMA'+str(EMA_list[1])]
    df_stock['MACD_signal'] = df_stock['MACD'].ewm(span=Signal).mean()
    df_stock['MACD_diff'] = df_stock['MACD']-df_stock['MACD_signal']

    #移動相関係数を計算する。

    #print(type(df_stock))
    #print(df_stock.columns)
    #print(df_stock)
    #ローソク足のチャートを作成する。
    inc = df_stock.Close > df_stock.Open
    dec = df_stock.Open > df_stock.Close
    w = 12*60*60*1000 # half day in ms
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    p = figure(x_axis_type='datetime', tools=TOOLS, plot_width=1000, title = securitycode+' Candlestick')
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3
    p.segment(df_stock.index, df_stock.High, df_stock.index, df_stock.Low, color="black")
    p.vbar(df_stock.index[inc], w, df_stock.Open[inc], df_stock.Close[inc], fill_color="#D5E1DD", line_color="black")
    p.vbar(df_stock.index[dec], w, df_stock.Open[dec], df_stock.Close[dec], fill_color="#F2583E", line_color="black")
    p.line(df_stock.index,df_stock['MA'+str(MA_list[0])+'d'] ,color = 'red',legend = str(MA_list[0])+'日移動平均')
    p.line(df_stock.index,df_stock['MA'+str(MA_list[1])+'d'] ,color = 'green',legend = str(MA_list[1])+'日移動平均')
    p.line(df_stock.index,df_stock['MA'+str(MA_list[2])+'d'] ,color = 'blue',legend = str(MA_list[2])+'日移動平均')
    p.line(df_stock.index,df_stock.BBpluse ,color = 'purple',legend = 'ボリンジャーバンド+')
    p.line(df_stock.index,df_stock.BBminus ,color = 'purple',legend = 'ボリンジャーバンド-')
    p.legend.location = 'top_left'

    # 移動平均乖離率のグラフ化    
    p1 = figure(width = 1000 , height = 200,x_axis_type = 'datetime' , tools=TOOLS ,x_range = p.x_range)
    p1.line(df_stock.index,df_stock['KairiMA'+str(MA_list[0])+'d'] ,color = 'green' ,legend = str(MA_list[0])+'日移動平均乖離率')
    p1.line(df_stock.index,0,color = 'red',line_width = 2)
    # 判例ラベルの位置設定
    p1.legend.location = 'top_left'

    #RSIのグラフ化
    p2 = figure(width = 1000 , height = 200,x_axis_type = 'datetime' , tools=TOOLS ,x_range = p.x_range)
    p2.line(df_stock.index,df_stock['rsi'] ,color = 'sandybrown' ,legend = 'RSI('+str(rsi_kikan)+')')
    p2.line(df_stock.index,70,color = 'red',line_width = 1)
    p2.line(df_stock.index,50,color = 'lightsteelblue',line_width = 1)
    p2.line(df_stock.index,30,color = 'red',line_width = 1)
    # 判例ラベルの位置設定
    p2.legend.location = 'top_left'

    #MACDのグラフ化
    p3 = figure(width = 1000 , height = 200,x_axis_type = 'datetime' , tools=TOOLS ,x_range = p.x_range)
    p3.line(df_stock.index,df_stock['MACD'] ,color = 'green' ,legend = 'MACD('+str(EMA_list[0])+'-'+str(EMA_list[1])+')')
    p3.line(df_stock.index,df_stock['MACD_signal'] ,color = 'red' ,legend = 'Signal('+str(Signal)+')')
    # 判例ラベルの位置設定
    p3.legend.location = 'top_left'

    #volumeの棒グラフを作成する
    p4 = figure(width = 1000 , height = 200,x_axis_type = 'datetime' , tools=TOOLS ,x_range = p.x_range)
    p4.vbar(df_stock.index[inc], w, df_stock.Volume[inc] , fill_color="#D5E1DD", line_color="green",legend ='VolumeInc')
    p4.vbar(df_stock.index[dec], w, df_stock.Volume[dec], fill_color="#F2583E", line_color="red",legend ='VolumeDec')
    # 判例ラベルの位置設定
    p4.legend.location = 'top_left'

    #/日経平均(ETF 1321)のグラフを作成する。


    #ファイルの保存と表示
    output_file('./output/stock_'+securitycode+'_bokeh.html', title=securitycode+"株価分析")
    show(column(p,p1,p2,p3,p4))  # open a browser
    #連続変換用のDelay
    time.sleep(1)


code       int64
name      object
sector    object
dtype: object
9984
yes


<ipython-input-2-57773967bc8f>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_up[df_stock.rsi_up < 0]=0
<ipython-input-2-57773967bc8f>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_down[df_stock.rsi_down < 0]=0


9434
yes


<ipython-input-2-57773967bc8f>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_up[df_stock.rsi_up < 0]=0
<ipython-input-2-57773967bc8f>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_down[df_stock.rsi_down < 0]=0


4502
yes


...

<ipython-input-2-57773967bc8f>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_up[df_stock.rsi_up < 0]=0
<ipython-input-2-57773967bc8f>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_down[df_stock.rsi_down < 0]=0


1605


<ipython-input-2-57773967bc8f>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_up[df_stock.rsi_up < 0]=0
<ipython-input-2-57773967bc8f>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_down[df_stock.rsi_down < 0]=0


8306


<ipython-input-2-57773967bc8f>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_up[df_stock.rsi_up < 0]=0
<ipython-input-2-57773967bc8f>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.rsi_down[df_stock.rsi_down < 0]=0
